# Chain Using LangGraph

In this section, we will see how we can build a simple chain using LangGraph that uses four important concepts:

- How to use chat messages as our graph state
- How to use chat models in graph nodes
- How to bind tools to our LLM in chat models
- How to execute the tools call in our graph nodes

In [1]:
from dotenv import load_dotenv
load_dotenv()

import os
os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")

## How to use chat messages as our graph state

### Messages

We can use messages to capture different roles within a conversation.  
LangChain provides various message types, including `HumanMessage`, `AIMessage`, `SystemMessage`, and `ToolMessage`.

These represent:
- A message from the user
- A message from the chat model
- A system message used to instruct the model’s behavior
- A message coming from a tool call

Every message has these important components:

- **content** — the content of the message
- **name** — specifies the name of the author
- **response_metadata** — optionally, a dictionary of metadata (e.g., often populated by the model provider for `AIMessage`s)

In [4]:
from langchain_core.messages import AIMessage,HumanMessage
from pprint import pprint

messages=[AIMessage(content=f"Please tell me how can I help",name="LLMModel")]
messages.append(HumanMessage(content=f"I want to learn coding",name="Giovanni"))
messages.append(AIMessage(content=f"Which programming language you want to learn",name="LLMModel"))
messages.append(HumanMessage(content=f"I want to learn python programming language",name="Giovanni"))

for message in messages:
    message.pretty_print()


================================== Ai Message ==================================
Name: LLMModel

Please tell me how can I help
================================ Human Message =================================
Name: Giovanni

I want to learn coding
================================== Ai Message ==================================
Name: LLMModel

Which programming language you want to learn
================================ Human Message =================================
Name: Giovanni

I want to learn python programming language


## Chat Models

We can use a sequence of messages as input to chat models using LLMs and OpenAI.

In [8]:
from langchain_groq import ChatGroq
llm=ChatGroq(model="meta-llama/llama-4-scout-17b-16e-instruct")
result=llm.invoke(messages)

In [9]:
result.response_metadata

{'token_usage': {'completion_tokens': 520,
  'prompt_tokens': 51,
  'total_tokens': 571,
  'completion_time': 1.189938867,
  'completion_tokens_details': None,
  'prompt_time': 0.000287286,
  'prompt_tokens_details': None,
  'queue_time': 0.091515252,
  'total_time': 1.190226153},
 'model_name': 'meta-llama/llama-4-scout-17b-16e-instruct',
 'system_fingerprint': 'fp_79da0e0073',
 'service_tier': 'on_demand',
 'finish_reason': 'stop',
 'logprobs': None,
 'model_provider': 'groq'}

## Tools

Tools can be integrated with LLM models to interact with external systems. External systems can be APIs or third-party tools.

Whenever a query is made, the model can choose to call a tool. This decision is based on the natural language input, and the tool call will return an output that matches the tool’s schema.